# Redes neuronales

Se utilizó el dataset <b>Fasion MNIST</b> de https://www.kaggle.com/zalando-research/fashionmnist

## Import de librerías

In [9]:
import numpy as np
import pandas as pd
from functools import reduce
from scipy import optimize as op

In [10]:
from RedesNeuronales import *
from utilities import *

In [11]:
import time

## Lectura de CSV

In [12]:
dataset = pd.read_csv('fashion-mnist_train.csv')
datasetTest = pd.read_csv('fashion-mnist_test.csv')

In [13]:
dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
datasetTest.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,2,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data = dataset.iloc[:, :].values
train, _, _ = getPartitions(data, 1, 0)

In [16]:
dataTest = datasetTest.iloc[:, :].values
test, _, _ = getPartitions(dataTest, 1, 0)

## Datos de entrenamiento y cross validation

In [17]:
y_train = train[:, 0].reshape(-1, 1)
x_train = train[:, 1:]

In [18]:
y_test = test[:, 0].reshape(-1, 1)
x_test = test[:, 1:].reshape(-1, 784)

In [33]:
NEURAL_NET = np.array([
    x_test.shape[1],
    130,
    10
])

theta_shapes = np.hstack((
    NEURAL_NET[1:].reshape(len(NEURAL_NET) - 1, 1),
    (NEURAL_NET[:-1] + 1).reshape(len(NEURAL_NET) - 1, 1)
))

In [34]:
flat_thetas = flatten_list_of_arrays([
    np.random.rand(*theta_shape)
    for theta_shape in theta_shapes
])

X = x_test / 1000.0
Y = (y_test == np.array(range(10))).astype(int)

In [35]:
flat_thetas = readFile("final.txt")

In [36]:
Thetas = inflate_matrixes(flat_thetas, theta_shapes)

In [37]:
a = feed_forward(Thetas, x_test)

In [38]:
success = 0
for i in range(len(a[-1])):
    value = np.where(np.isclose(a[-1][i], max(a[-1][i])))[0][0]
    
    if (value == y_test[i][0]):
        success += 1
        
print("Success: ", success, success * 100/len(a[-1]))

Success:  7453 74.53
